In [1]:
!pip install llama_cloud_services openpyxl


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import pandas as pd
from llama_cloud_services import LlamaExtract
from financial_schemas_incomestat import IncomeStatement_2024_25  #This could be adjusted through schemas.py
#from dotenv import load_dotenv


In [3]:
PDF_ROOT = "scrapping/university_pdfs"
OUTPUT_ROOT = "output_fin_incomestatement_nk_v1"
os.makedirs(OUTPUT_ROOT, exist_ok=True)  
AGENT_ID = "56843d2c-7e9b-445d-b634-9833dd1cb4db" #Different based on your LLamaCloud account
api_key = os.getenv("LLAMACLOUD_API_KEY")

In [4]:
extractor = LlamaExtract(
    api_key="llx-63CU3PdyDo0d230ureocmy9JOHgnPwYgE2HETi55DqzYCIpy",  
    project_id="8c10e62e-3810-4193-915d-d2d11105826d"
)



#agent = extractor.create_agent(name = "endowment-parser-2024", data_schema=EndowmentAndInvestmentLevels_2024_25)

agent = extractor.get_agent(id = AGENT_ID)

#uncomment the following lines if you updated the schema
agent.data_schema = IncomeStatement_2024_25.schema()
agent.save()
agent = extractor.get_agent(id = AGENT_ID)


/var/folders/m1/8j3nd3m95y1brb0j52fx43fr0000gn/T/ipykernel_30923/2778103400.py:13: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  agent.data_schema = IncomeStatement_2024_25.schema()


In [5]:
agent.data_schema

{'additionalProperties': False,
 'properties': {'gross_tuition_revenue': {'anyOf': [{'type': 'integer'},
    {'type': 'null'}],
   'description': "Gross tuition revenue before financial aid for 2024–2025. Look for labels like 'Gross Tuition and Fees'. Exclude net tuition or post-discounted amounts."},
  'financial_aid': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
   'description': "Institutional financial aid or tuition discounts for 2024–2025. Labeled 'Scholarships', 'Grants', or 'Financial Aid'. Do not include federal aid."},
  'net_tuition_revenue': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
   'description': "Tuition revenue net of financial aid for 2024–2025. Must be explicitly labeled as 'Net Tuition Revenue' or equivalent. Do not compute manually."},
  'federal_grants_contracts': {'anyOf': [{'type': 'integer'},
    {'type': 'null'}],
   'description': "Federal operating grants and contracts for 2024–2025. Look for 'Federal Grants & Contracts' in operating revenue 

The following two cell blocks extract all schools' info into one excel file per school.

In [7]:
def process_school(school_name, school_dir):
    combined   = {}
    first_keys = None

    for fname in sorted(os.listdir(school_dir)):
        if not fname.lower().endswith(".pdf"):
            continue
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v
        except Exception as err:
            print(f"Skipped {fname}: {err}")

    if first_keys:
        df = pd.DataFrame.from_dict(combined, orient="index", columns=["2024-25"])
        df.index.name = "Metric"
        outfile = os.path.join(OUTPUT_ROOT, f"{school_name}.xlsx")
        df.to_excel(outfile)
        print(f"Saved output to {outfile}")
    else:
        print(f"No PDF data found for {school_name}")

In [ ]:
# Loop over schools
for school in sorted(os.listdir(PDF_ROOT)):
    school_dir = os.path.join(PDF_ROOT, school)
    if not os.path.isdir(school_dir):
        continue
    print(f"Processing school: {school}")
    process_school(school, school_dir)

print("Extraction complete.")

Processing school: ARIZONA_STATE_UNIVERSITY
Extracting data from FY_2024_Arizona_State_University_Annual_Comprehensive_Financial_Report_for_the_year_ended_06_30_2024__4.1_MB_.pdf


Extracting files: 100%|██████████| 1/1 [00:24<00:00, 24.88s/it]


Extracting data from FY_2024_Arizona_State_University_Continuing_Disclosure_Undertaking_for_the_year_ended_06_30_2024__160_KB_.pdf


Extracting files: 100%|██████████| 1/1 [00:12<00:00, 12.11s/it]


Saved output to output_fin_incomestatement_nk_v1/ARIZONA_STATE_UNIVERSITY.xlsx
Processing school: BRADLEY_UNIVERSITY
Extracting data from Annual_Financial_Information_and_Operating_Data__Rule_15c2-12__for_FY24_for_the_year_ended_05_31_2024__227_KB_.pdf


...

Extracting files: 100%|██████████| 1/1 [01:29<00:00, 89.86s/it]


Extracting data from Annual_Financial_Information_and_Operating_Data_for_the_year_ended_06_30_2024_Document2__315_KB_.pdf


Extracting files: 100%|██████████| 1/1 [00:18<00:00, 18.28s/it]

Saved output to output_fin_incomestatement_nk_v1/UNIVERSITY_OF_MINNESOTA.xlsx
Extraction complete.


Extracting files: 100%|██████████| 1/1 [00:18<00:00, 18.03s/it]


The following cell block extracts all the schools' info into one excel sheet but in different tabs.

In [9]:
OUTPUT_FILE = os.path.join(OUTPUT_ROOT, "all_schools.xlsx")

writer = pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl")

for school in sorted(os.listdir(PDF_ROOT)):
    school_dir = os.path.join(PDF_ROOT, school)
    if not os.path.isdir(school_dir):
        continue

    combined   = {}
    first_keys = None
    for fname in sorted(os.listdir(school_dir)):
        if not fname.lower().endswith(".pdf"):
            continue
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {school}/{fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v
        except Exception as err:
            print(f"Skipped {fname}: {err}")

    if first_keys:
        df = pd.DataFrame.from_dict(combined, orient="index", columns=["2024-25"])
        df.index.name = "Metric"
        sheet_name = school[:31]
        df.to_excel(writer, sheet_name=sheet_name)
    else:
        print(f"No data for {school}.")

writer.close()
print(f"All schools written to {OUTPUT_FILE}")

Extracting data from ARIZONA_STATE_UNIVERSITY/FY_2024_Arizona_State_University_Annual_Comprehensive_Financial_Report_for_the_year_ended_06_30_2024__4.1_MB_.pdf
Extracting data from ARIZONA_STATE_UNIVERSITY/FY_2024_Arizona_State_University_Continuing_Disclosure_Undertaking_for_the_year_ended_06_30_2024__160_KB_.pdf
Extracting data from BRADLEY_UNIVERSITY/Annual_Financial_Information_and_Operating_Data__Rule_15c2-12__for_FY24_for_the_year_ended_05_31_2024__227_KB_.pdf
Extracting data from BRADLEY_UNIVERSITY/Audited_Financial_Statements_or_ACFR__Rule_15c2-12__for_FY24_for_the_year_ended_05_31_2024__541_KB_.pdf
Extracting data from CALIFORNIA_STATE_UNIVERSITY/Exhibit_1_CSU_Combined_Financial_Stmts_6-30-24_for_the_year_ended_06_30_2024__15.6_MB_.pdf
Extracting data from CALIFORNIA_STATE_UNIVERSITY/Exhibit_1_CSU_Combined_Financial_Stmts_6-30-24_for_the_year_ended_06_30_2024__308_KB_.pdf
Extracting data from CALIFORNIA_STATE_UNIVERSITY/Exhibit_1_DRAFT_UNAUDITED_CSU_Combined_Financial_Statemen

In [10]:
#Combine all the tabs into one sheet if wanted
file_path   = "output_fin_incomestatement_nk_v1/all_schools.xlsx"
output_path = "output_fin_incomestatement_nk_v1/all_schools_combined.xlsx"

raw = pd.read_excel(file_path, sheet_name=None, index_col=0)

school_series = {
    school: df.iloc[:, 0]                      # first (only) value column
    for school, df in raw.items()
}

df_comb = pd.DataFrame(school_series).T
df_comb.index.name = "School"                 
df_comb.insert(0, "Year", "2023‑2024")

#df_comb.loc['Texas_A&M', ['Total_Headcount','Undergraduate_Headcount']] = \
    #df_comb.loc['Texas_A&M', ['Undergraduate_Headcount','Total_Headcount']].values

# df_comb.loc['California_state_university', 'Undergraduate_Headcount'] = None 
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    df_comb.to_excel(writer, sheet_name="Combined")

print("Saved:", output_path)

Saved: output_fin_incomestatement_nk_v1/all_schools_combined.xlsx
